In [1]:
# 词典库
vocab = set([line.rstrip() for line in open('vocab.txt')])

In [2]:
print(vocab)

{'sidechairs', 'Reverently', 'scarcity', 'circulate', 'dearest', 'sharks', 'stoves', 'Hair', 'complained', 'empathy', 'Richards', 'comets', 'signifies', 'sobriquet', 'Osaka', 'occupying', 'micrometeorite', 'unequalled', 'accountability', 'sera', 'immobility', 'conservatives', 'spy', 'configuration', 'Duncan', 'firmer', 'strapping', 'Conneaut', 'Cudmore', 'goddamned', 'Coeditors', 'leafiest', 'applause', 'Expenditure', 'dockside', 'Langer', 'ranchers', 'sixteen', 'Penny', 'deprecatory', 'spare', 'blending', 'Greenberg', 'Bostonian', 'eruption', 'Abernathys', 'refined', 'schema', 'slacking', 'VW', 'Emile', 'chased', 'equanimity', 'fluent', 'untouched', 'longitudinal', 'erase', 'drumming', 'Americana', 'discourses', 'Consent', 'slopes', 'heal', 'Kepler', 'Petey', 'rescheduling', 'Paulus', 'numerological', 'Responsibility', 'blandness', 'Deterioration', 'cuirassiers', 'perforce', 'noiseless', 'subverting', 'Believe', 'quoting', 'naivete', 'Mason', 'farmer', 'Sie', 'misdeeds', 'arroyo', 'Un

In [8]:
# 需要生成所有首选集合
def generate_candidates(word):
    """
    word: 给定的输入（错误的输入）
    返回所有（valid）候选集合
    """
    # 生成编辑距离为1的单词
    # 1 insert 2 delete 3 replace
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word)+1)]
    # insert操作
    inserts = [L+c+R for L, R in splits for c in letters]
    # delete
    deletes = [L+R[1:] for L,R in splits if R]
    # replace
    replaces = [L+c+R[1:] for L,R in splits if R for c in letters]
    candidates = set(inserts+deletes+replaces)
    # 过滤掉不存在于词典库里面的单词
    return [word for word in candidates if word in vocab] 
generate_candidates("apple")

['apply', 'apple', 'ample', 'apples']

In [9]:
from nltk.corpus import reuters

# 读取语料库
categories = reuters.categories()
corpus = reuters.sents(categories=categories)

In [10]:
# 构建语言模型: bigram
term_count = {}
bigram_count = {}
for doc in corpus:
    doc = ['<s>'] + doc
    for i in range(0, len(doc)-1):
        # bigram: [i,i+1]
        term = doc[i]
        bigram = doc[i:i+2]
        
        if term in term_count:
            term_count[term]+=1
        else:
            term_count[term]=1
        bigram = ' '.join(bigram)
        if bigram in bigram_count:
            bigram_count[bigram]+=1
        else:
            bigram_count[bigram]=1


In [11]:
print(term_count)

{'<s>': 54716, 'ASIAN': 12, 'EXPORTERS': 46, 'FEAR': 2, 'DAMAGE': 13, 'FROM': 208, 'U': 6388, '.': 45900, 'S': 6382, '.-': 167, 'JAPAN': 295, 'RIFT': 1, 'Mounting': 1, 'trade': 2271, 'friction': 28, 'between': 1053, 'the': 58251, 'And': 227, 'Japan': 1594, 'has': 4679, 'raised': 334, 'fears': 65, 'among': 229, 'many': 297, 'of': 35979, 'Asia': 71, "'": 11272, 's': 9298, 'exporting': 51, 'nations': 399, 'that': 7377, 'row': 49, 'could': 1431, 'inflict': 1, 'far': 275, '-': 13705, 'reaching': 37, 'economic': 784, 'damage': 128, ',': 72359, 'businessmen': 56, 'and': 25043, 'officials': 836, 'said': 25221, 'They': 562, 'told': 1424, 'Reuter': 31, 'correspondents': 7, 'in': 26478, 'Asian': 53, 'capitals': 7, 'a': 23492, 'Move': 1, 'against': 1428, 'might': 399, 'boost': 203, 'protectionist': 86, 'sentiment': 61, 'lead': 261, 'to': 34035, 'curbs': 36, 'on': 8556, 'American': 817, 'imports': 776, 'their': 1345, 'products': 703, 'But': 1157, 'some': 1198, 'exporters': 195, 'while': 812, 'confl

In [12]:
# 用户打错的概率统计 - channel probility

In [13]:
channel_prob = {}
for line in open('spell-errors.txt'):
    items = line.split(":")
    correct = items[0].strip()
    mistakes = [item.strip() for item in items[1].strip().split(",")]
    channel_prob[correct] = {}
    for mis in mistakes:
        channel_prob[correct][mis] = 1.0/len(mistakes)

In [14]:
V = len(term_count.keys())

In [15]:
import numpy as np
V = len(term_count.keys())

file = open("testdata.txt", 'r')
for line in file:
    items = line.rstrip().split('\t')
    line = items[2].split()
    # line = ["I", "like", "playing"]
    for word in line:
        if word not in vocab:
            # 需要替换word成正确的单词
            # Step1: 生成所有的(valid)候选集合
            candidates = generate_candidates(word)
            
            # 一种方式： if candidate = [], 多生成几个candidates, 比如生成编辑距离不大于2的
            # TODO ： 根据条件生成更多的候选集合
            if len(candidates) < 1:
                continue   # 不建议这么做（这是不对的） 
            probs = []
            # 对于每一个candidate, 计算它的score
            # score = p(correct)*p(mistake|correct)
            #       = log p(correct) + log p(mistake|correct)
            # 返回score最大的candidate
            for candi in candidates:
                prob = 0
                # a. 计算channel probability
                if candi in channel_prob and word in channel_prob[candi]:
                    prob += np.log(channel_prob[candi][word])
                else:
                    prob += np.log(0.0001)
                
                # b. 计算语言模型的概率
                idx = items[2].index(word)+1
                if items[2][idx - 1] in bigram_count and candi in bigram_count[items[2][idx - 1]]:
                    prob += np.log((bigram_count[items[2][idx - 1]][candi] + 1.0) / (
                            term_count[bigram_count[items[2][idx - 1]]] + V))
                # TODO: 也要考虑当前 [word, post_word]
                #   prob += np.log(bigram概率)
                
                else:
                    prob += np.log(1.0 / V)

                probs.append(prob)
                
            max_idx = probs.index(max(probs))
            print (word, candidates[max_idx])

protectionst protectionist
products. products
long-run, long-run
gain. gains
17, 17e
retaiation retaliation
cost. costs
busines, business
ltMC.T. ltMC.T
U.S., U.S.
Murtha, Murtha
worried. worried
seriousnyss seriousness
aganst against
us, us
named. named
year, yeard
sewll sell
dlrs, dlrs
world's worlds
largest. largest
markets, markets
importsi imports
Products, Products
Retaliation, Retaliation
Group. Group
Korea's Koreans
Korea, Korea
Japan. Japan
Koreva Korea
U.S., U.S.
1985. 1985
Malaysia, Malaysia
Japn Japan
Kong, Kong
view. view
advantagne advantage
imports. imports
view, view
Lawrenc Lawrence
Mills, Mills
Industry. Industry
imports, imports
sources. sources
disadxantage disadvantage
trade, trade
said. said
market, markets
Friday. Fridays
said. said
cenntred centred
beef, beefy
country. country
trad trade
continue. continue
Liberala Liberal
economy. economy
inoclude include
year. yeard
program. program
Representetive Representative
Kuroda, Kuroda
MITI, MITI
dispute. disputes
tonn

state's states
workers' workers
laws. laws
Lammers, Lammers
Queen's Queens
Flevoland, Flevoland
redundancies. redundancies
employers' employers
proposas proposes
year. yeard
court's courts
interom interim
grounds. grounds
conplicated complicated
future. future
Meanwhile, Meanwhile
port's ports
continued, continued
strike, strike
affected, affected
Mij, Mij
larest largest
sector, sectors
said. said
writte write
tomorrow. tomorrow
one, one
pct, pct
whilen while
pct. pct
addition, addition
thre thru
pct. pct
bringsi brings
stg. stg
Todaay Today
dlrs, dlrs
dlrs. dlrs
praetax pretax
dlrs. dlrs
1986, 1986
figeure figure
pct. pct
Canion, Canion
Compaq, Compaq
peniod period
31, 31
analysts' analysts
dlrs. dlrs
share. shares
dlrs, dlrs
share, shares
rifst rift
1986. 1986
386, 386
increase. increases
computers, computers
Cawion Canion
said. said
demad dead
quarter, quarter
month. months
I.U. I.U
ltHE. ltHE
said. said
executiocn execution
agreements, agreements
involved, involved
I.U. I.U
Interna

repiorted reported
trading, trading
period. periods
1986, 1986
becausel because
securities. securities
quarter, quarter
falls, falls
Euromarket. Euromarket
side, sided
Foreigm Foreign
mln. mln
income, incomes
commissions, commissions
mln, mln
mln. mln
quarter, quarter
earlier, earlier
diely diety
pct. pct
non-accrual, non-accrual
weich which
received, received
pct. pct
payments, payments
dlrs, dlrs
added. added
Afer After
losses, losses
earlier. earlier
mln, mln
earlier. earlier
Exacluding Excluding
Brazil, Brazil
quarter's quarters
dlrs, dlrs
earlier. earlier
dlrs, dlrs
increae increase
costs, costs
said. said
Leonaerdo Leonardo
Brito, Briton
Brasilia, Brasilia
year's years
lost. lost
problems. problems
tonnes, tonnes
sufficienti sufficient
crop. crop
Brio trio
distributed. distributed
Paulo, Paulo
betwein between
nation's nations
capacity, capacity
production. productions
regions, regions
wheer sheer
capacity. capacity
Centre-West, Centre-West
maize, maize
nation's nations
space. spa

1986. 1986
hodding rodding
companya company
Ecuador. Ecuador
inaterest interest
biluion billion
20. 20
Ecuador, Ecuador
whic chic
dlrs, dlrs
interst interest
Mirch birch
Ecuadr Ecuador
oil, oils
revenues. revenues
announcement, announcements
Securitye Security
receiverd received
cash. cash
Brzil Broil
ratel rates
1987. 1987
negotiativons negotiations
resme resume
bilion billion
15. 15
significantr significant
continuingn continuing
foilefd foiled
Basra, Basra
unit, unite
attacked. attacked
warplanes, warplanes
hilicopter helicopter
tanks. tanks
advnance advance
occupiede occupied
said. said
wouded worded
theirf their
druing drying
today. today
denmied denied
reporn reborn
down. downs
vanal banal
Gulf. Gulf
Iriqi Iraqi
unitsx units
attmpting attempting
terminal, terminal
said. said
Chancelilor Chancellor
meeing geeing
Treasury. Treasury
Sevin hevin
gatheru gather
Treasury. Treasury
Houston, Houston
Service, Services
FSIS, FSIS
pmoposed proposed
woubd would
port. ports
phades phases
disu